Import the `Branch` module.

In [1]:
from timeflux.core.branch import Branch

Currently, the `Branch` module simulates an online application and needs a reference timestamp. This will be simplified in future versions.

In [2]:
import time
from timeflux.core.registry import Registry
Registry.cycle_start = time.time()

Define the graph as a Python dictionary. This is a an exact representation of a Timeflux YAML graph.

In [3]:
graph = {
    'nodes': [
        {
            'id': 'node_1',
            'module': 'timeflux.nodes.random',
            'class': 'Random',
            'params': {
                'seed': 1,
                'rows_min': 3,
                'rows_max': 3,
                'columns': 4
            }
        },
        {
            'id': 'node_2',
            'module': 'timeflux_example.nodes.arithmetic',
            'class': 'Add',
            'params': {
                'value': 1
            }
        },
        {
            'id': 'node_3',
            'module': 'timeflux_example.nodes.arithmetic',
            'class': 'Add',
            'params': {
                'value': 2
            }
        },        
    ],
    'edges': [
        {
            'source': 'node_1',
            'target': 'node_2'
        },
        {
            'source': 'node_1',
            'target': 'node_3'
        }        
    ]
}

Instantiate and run the branch once.

In [4]:
branch = Branch(graph)
branch.run()

Inspect the output of the second node.

In [5]:
branch.get_port('node_2', 'o').data

,0,1,2,3
2023-08-16 15:32:33.969455,6,9,10,6
2023-08-16 15:32:34.302788,1,1,2,8
2023-08-16 15:32:34.636121,7,10,3,5


Inspect the output of the first node. Notice that the values are exactly the same as the second node. This is because Timeflux creates a deep copy of objects only when absolutely required.

In [6]:
branch.get_port('node_1', 'o').data

,0,1,2,3
2023-08-16 15:32:33.969455,6,9,10,6
2023-08-16 15:32:34.302788,1,1,2,8
2023-08-16 15:32:34.636121,7,10,3,5


This can be confirmed by checking the IDs of objects.

In [7]:
print(id(branch.get_port('node_1', 'o').data))
print(id(branch.get_port('node_2', 'o').data))

4936981696
4936981696


But when there is a fork in the graph, the data is actually copied, and we can confirm that everything is working well :)

In [8]:
branch.get_port('node_3', 'o').data

,0,1,2,3
2023-08-16 15:32:33.969455,7,10,11,7
2023-08-16 15:32:34.302788,2,2,3,9
2023-08-16 15:32:34.636121,8,11,4,6


In [9]:
print(id(branch.get_port('node_3', 'o').data))

4936980832
